# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [26]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# One-hot encode categorical variables
label_encoder = LabelEncoder()
crops["crop"] = label_encoder.fit_transform(crops["crop"])

# Separate features and target variables
df_features = crops.iloc[:,:4]
target = "crop"

# Dictionary to store best feature
feature_accuracies = {}

# Hyperparameter tuning using GridSearchCV
param_grid = {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000], # Regularization strength
    "penalty" : ["l1", "l2"], # Type of Regularization
    "solver" : ["saga", "liblinear"], # Both solvers support L1 and L2 regularization
}

# Train Logistic Regression Model
for feature in df_features:
    X = crops[[feature]]
    y = crops[target]
    
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Hyperparameter Tuning
    grid_search = GridSearchCV(LogisticRegression(max_iter=150), param_grid, cv=5, scoring="accuracy")
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    
    # Predict and Evaluate
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Store accuracy for the feature
    feature_accuracies[feature] = accuracy
    
# Find the best feature
best_predictive_feature = {
    max(feature_accuracies, key=feature_accuracies.get) : max(feature_accuracies.values())
}

# Print results
print("Best Predictive Feature:", best_predictive_feature)